# SQL in Python - Connecting to and retrieving data from PostgreSQL

Previously, you have learned how to connect to a SQL database by using a SQL client such as DBeaver. Apart from connecting to databases, DBeaver also allows you to run SQL queries against the database, create new tables and populate them with data as well as retrieving the data.

Python also allows executing SQL queries and getting the result into a Python object, for example a Pandas data frame. Instead of exporting a .csv file from DBeaver you can directly get the data you need into Python and continue your work. In addition we can reduce the steps by connecting to the database from Python directly, eliminating the need for a separate SQL client.

After you have the data in Python in the required shape you can export the data into a .csv file. This file is for your own reference, please avoid sending .csv files around - database is the point of reference when it comes to data. 

Having a copy of a .csv file (or another format) can speed up your analysis work. Imagine that the query takes 25 minutes to run. If you made some mistakes in your Python code you might need to go back to the original dataset. Instead of having to rerun the SQL query and having to wait you can read in the .csv file you have previously saved on your hard disk into Python and continue with your analysis work. 

**In this notebook you will see 2 ways to connect to SQL-Databases and export the data to a CSV file**


## Creating a connection to a PostgreSQL database with Python

There are 2 python packages that are the "go-to" when it comes to connecting to SQL-Databases: `psycopg2` and `sqlalchemy` 

### Connecting via psycopg2

In [51]:
import pandas as pd
import psycopg2


In order to create a connection to our PostgreSQL database we need the following information:

- host = the address of the machine the database is hosted on
- port = the virtual gate number through which communication will be allowed
- database = the name of the database
- user = the name of the user
- password = the password of the user

Because we don't want that the database information is published on github we put it into a `.env` file which is added into the `.gitignore`. 
In these kind of files you can store information that is not supposed to be published.
With the `dotenv` package you can read the `.env` files and get the variables.
(We will share the file with you on Slack!)


In [7]:
!pip install python-dotenv

In [63]:
import os
from dotenv import load_dotenv

load_dotenv()

DATABASE = os.getenv('DATABASE')
USER_DB = os.getenv('USER_DB')
PASSWORD = os.getenv('PASSWORD')
HOST = os.getenv('HOST')
PORT = os.getenv('PORT')

The function from the psycopg2 package to create a connection is called `connect()`.
`connect()` expects the parameters listed above as input in order to connect to the database.

In [53]:
# Create connection object conn
conn = psycopg2.connect(
    database=DATABASE,
    user=USER_DB,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

### Retrieving data from the database with psycopg2

Before we can use our connection to get data, we have to create a cursor. A cursor allows Python code to execute PostgreSQL commmands in a database session.
A cursor has to be created with the `cursor()` method of our connection object conn.

In [60]:
cur = conn.cursor()

Now we can run SQL-Queries with `cur.execute('QUERY')` and then run `cur.fetchall()` to get the data:

In [25]:
cur.execute("""
SELECT 
    s.date,
    s.price,
    s.house_id,
    d.bedrooms,
    d.sqft_living,
    d.view,
    d.grade,
    d.yr_renovated,
    d.zipcode,
    d.waterfront 
FROM eda.king_county_house_details d 
LEFT JOIN eda.king_county_house_sales s
    ON s.house_id = d.id
WHERE d.yr_renovated > 0 
  AND d.bedrooms > 2 
  AND d.sqft_living > 2500 
  AND s.price > 750000;
""")
cur.fetchall()

[(datetime.date(2014, 9, 8),
  905000.0,
  1483300570,
  4.0,
  3300.0,
  0.0,
  7,
  19910,
  98040,
  0.0),
 (datetime.date(2014, 10, 7),
  1090000.0,
  2450000295,
  3.0,
  2920.0,
  0.0,
  8,
  20100,
  98004,
  0.0),
 (datetime.date(2015, 3, 2),
  1450000.0,
  4389200955,
  4.0,
  2750.0,
  0.0,
  8,
  19920,
  98004,
  0.0),
 (datetime.date(2015, 2, 21),
  1510000.0,
  46100204,
  5.0,
  3300.0,
  3.0,
  9,
  19910,
  98040,
  None),
 (datetime.date(2015, 5, 13),
  2050000.0,
  7424700045,
  5.0,
  3830.0,
  1.0,
  9,
  19940,
  98122,
  0.0),
 (datetime.date(2014, 9, 16),
  880000.0,
  3179100060,
  4.0,
  2800.0,
  0.0,
  9,
  20020,
  98105,
  None),
 (datetime.date(2014, 6, 20),
  1950000.0,
  5486800070,
  7.0,
  4640.0,
  1.0,
  11,
  20030,
  98040,
  None),
 (datetime.date(2014, 8, 19),
  1750000.0,
  7533800295,
  4.0,
  3460.0,
  1.0,
  10,
  19980,
  98115,
  0.0),
 (datetime.date(2014, 6, 25),
  995000.0,
  7636800041,
  3.0,
  4380.0,
  3.0,
  9,
  19900,
  98166,
  

With `conn.close()` you can close the connection again.

In [55]:
#close the connection
conn.close()

But we want to work with the data. The easiest way is to import the data into pandas dataframes. We can use `pd.read_sql_query` or `pd.read_sql_table` or for convenience `pd.read_sql`.

This function is a convenience wrapper around read_sql_table and read_sql_query (for backward compatibility). It will delegate to the specific function depending on the provided input. A SQL query will be routed to read_sql_query , while a database table name will be routed to read_sql_table . Note that the delegated function might have more specific notes about their functionality not listed here.

In [58]:
# Open connection again because we closed it
conn = psycopg2.connect(
    database=DATABASE,
    user=USER_DB,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

In [61]:
cur.execute("""
SELECT 
    s.date,
    s.price,
    s.house_id,
    d.bedrooms,
    d.sqft_living,
    d.view,
    d.grade,
    d.yr_renovated,
    d.zipcode,
    d.waterfront 
FROM eda.king_county_house_details d 
LEFT JOIN eda.king_county_house_sales s
    ON s.house_id = d.id
WHERE d.yr_renovated > 0 
  AND d.bedrooms > 2 
  AND d.sqft_living > 2500 
  AND s.price > 750000;
""")
cur.fetchall()

[(datetime.date(2014, 9, 8),
  905000.0,
  1483300570,
  4.0,
  3300.0,
  0.0,
  7,
  19910,
  98040,
  0.0),
 (datetime.date(2014, 10, 7),
  1090000.0,
  2450000295,
  3.0,
  2920.0,
  0.0,
  8,
  20100,
  98004,
  0.0),
 (datetime.date(2015, 3, 2),
  1450000.0,
  4389200955,
  4.0,
  2750.0,
  0.0,
  8,
  19920,
  98004,
  0.0),
 (datetime.date(2015, 2, 21),
  1510000.0,
  46100204,
  5.0,
  3300.0,
  3.0,
  9,
  19910,
  98040,
  None),
 (datetime.date(2015, 5, 13),
  2050000.0,
  7424700045,
  5.0,
  3830.0,
  1.0,
  9,
  19940,
  98122,
  0.0),
 (datetime.date(2014, 9, 16),
  880000.0,
  3179100060,
  4.0,
  2800.0,
  0.0,
  9,
  20020,
  98105,
  None),
 (datetime.date(2014, 6, 20),
  1950000.0,
  5486800070,
  7.0,
  4640.0,
  1.0,
  11,
  20030,
  98040,
  None),
 (datetime.date(2014, 8, 19),
  1750000.0,
  7533800295,
  4.0,
  3460.0,
  1.0,
  10,
  19980,
  98115,
  0.0),
 (datetime.date(2014, 6, 25),
  995000.0,
  7636800041,
  3.0,
  4380.0,
  3.0,
  9,
  19900,
  98166,
  

In [49]:
#close the connection
conn.close()

In [67]:
query_string = """
SELECT 
    s.date,
    s.price,
    s.house_id,
    d.bedrooms,
    d.sqft_living,
    d.view,
    d.grade,
    d.yr_renovated,
    d.zipcode,
    d.waterfront 
FROM eda.king_county_house_details d 
LEFT JOIN eda.king_county_house_sales s
    ON s.house_id = d.id
WHERE d.yr_renovated > 0 
  AND d.bedrooms > 2 
  AND d.sqft_living > 2500 
  AND s.price > 750000;
"""

cur = conn.cursor()
cur.execute(query_string)
rows = cur.fetchall()

# Convert result to a DataFrame
columns = [desc[0] for desc in cur.description]
df_psycopg = pd.DataFrame(rows, columns=columns)


In [68]:
df_psycopg.head()

,date,price,house_id,bedrooms,sqft_living,view,grade,yr_renovated,zipcode,waterfront
0,2014-09-08,905000.0,1483300570,4.0,3300.0,0.0,7,19910,98040,0.0
1,2014-10-07,1090000.0,2450000295,3.0,2920.0,0.0,8,20100,98004,0.0
2,2015-03-02,1450000.0,4389200955,4.0,2750.0,0.0,8,19920,98004,0.0
3,2015-02-21,1510000.0,46100204,5.0,3300.0,3.0,9,19910,98040,NaN
4,2015-05-13,2050000.0,7424700045,5.0,3830.0,1.0,9,19940,98122,0.0


In [69]:
#export the data to a csv-file
df_psycopg.to_csv('data/eda.csv',index=False)

### Connecting and retrieving data via SQLAlchemy

`sqlalchemy` works similarly. Here you have to create an engine with the database sting (a link that includes every information we entered in the conn object)

In [ ]:
from sqlalchemy import create_engine

#read the database string from the .env
load_dotenv()

DB_STRING = os.getenv('DB_STRING')

db = create_engine(DB_STRING)

In [ ]:
from dotenv import load_dotenv

load_dotenv()

And then you can import that engine with a query into a pandas dataframe.

In [ ]:
#import the data to a pandas dataframe
query_string = """
SELECT 
    s.date,
    s.price,
    s.house_id,
    d.bedrooms,
    d.sqft_living,
    d.view,
    d.grade,
    d.yr_renovated,
    d.zipcode,
    d.waterfront 
FROM eda.king_county_house_details d 
LEFT JOIN eda.king_county_house_sales s
    ON s.house_id = d.id
WHERE d.yr_renovated > 0 
  AND d.bedrooms > 2 
  AND d.sqft_living > 2500 
  AND s.price > 750000;
"""

# Optional: Rollback any failed transaction before querying
conn.rollback()

# Use pandas to read the SQL query result
df_psycopg = pd.read_sql(query_string, conn)
df_sqlalchemy = pd.read_sql(query_string, db)


In [13]:
df_sqlalchemy.head()

,date,price,house_id,id
0,2014-10-13,221900.0,7129300520,1
1,2014-12-09,538000.0,6414100192,2
2,2015-02-25,180000.0,5631500400,3
3,2014-12-09,604000.0,2487200875,4
4,2015-02-18,510000.0,1954400510,5


Because we don't want to run the queries over and over again we can export the data into a .csv file in order to use it in other notebooks as well. 

In [ ]:
#export the data to a csv-file
df_sqlalchemy.to_csv('eda.csv',index=False)

In [18]:
#import the data from a csv-file
df_import = pd.read_csv('data/eda.csv')